In [ ]:
import pandas as pd
import numpy as np
import os

# Define the folder path
folder_path = 'GES16Aand16BStimulationMonitoringApril2024/16AStimulationCatalogues'

# Load all CSV files from the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f.startswith('FORGE')]

# Merge all CSV files into a single dataframe
df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]
merged_df = pd.concat(df_list, ignore_index=True)

# Inspect whether the column "Source" contains unique values
unique_sources = merged_df['Source'].is_unique
print(f"Column 'Source' contains unique values: {unique_sources}")

In [ ]:
merged_df

In [ ]:
merged_df['Source'].to_numpy()

In [ ]:
# Convert the 'Source' column to a numpy array
source_array = merged_df['Source'].to_numpy()

# Check if all values in the numpy array are unique
is_unique = len(np.unique(source_array)) == len(source_array)
print(f"All values in the numpy array are unique: {is_unique}")

In [ ]:
# Find duplicate values and their indices
duplicates = {}
for index, value in enumerate(source_array):
    if value in duplicates:
        duplicates[value].append(index)
    else:
        duplicates[value] = [index]

# Filter out values that are not duplicates
duplicates = {key: value for key, value in duplicates.items() if len(value) > 1}

print("Duplicate values and their indices:")
for value, indices in duplicates.items():
    print(f"Value: {value}, Indices: {indices}")

In [ ]:
merged_df[merged_df['Source']==9675]